In [1]:
import numpy as np
import pandas as pd
import urllib
import requests
import random
import ssl
import time
from bs4 import BeautifulSoup
import warnings
import tqdm
import pickle
from multiprocessing import Pool
warnings.simplefilter("ignore")

### Подгрузка всех № участников

In [2]:
%%time
# Позволяет обращаться к сайту без верификации безопасности. В противном случае не допускает к Казахскому сайту
context = ssl._create_unverified_context()
# Итерируемся по каждому page с № участников, пока не закончатся page
error_signal = None
page = 1
df_info = pd.DataFrame()
while error_signal is None:
    try:
        # Будем вытаскивать по-максимуму инфы с одной страницы, какую только позволяет сайт: по 2000 участников
        url = 'https://goszakup.gov.kz/ru/registry/supplierreg?count_record=2000&page=' + str(page)
        web_page = urllib.request.urlopen(url, context = context)
        # Распарсим информацию с сайта с помощью BeautifulSoup
        soup = BeautifulSoup(web_page, "html.parser", from_encoding = 'utf-8')
        table_find = soup.find('table', attrs={'class':'table table-bordered table-striped dataTable responsive'})
        all_td = table_find.findAll('td')
        for ind, val in enumerate(all_td):
            # Каждый пятый элемент в all_td содержит № участника: как раз то, что нам надо вытащить
            if ind % 5 == 0:
                # Используем strip, чтобы удалить все пробельные символы в строке. Номер участников будем сохранять в колонке id
                df_info = df_info.append({'id': val.text.strip()}, ignore_index = True)
        print('page {} processed'.format(page))
        page += 1
    except:
        error_signal = True

page 1 processed
page 2 processed
page 3 processed
page 4 processed
page 5 processed
page 6 processed
page 7 processed
page 8 processed
page 9 processed
page 10 processed
page 11 processed
page 12 processed
page 13 processed
page 14 processed
page 15 processed
page 16 processed
page 17 processed
page 18 processed
page 19 processed
page 20 processed
page 21 processed
page 22 processed
page 23 processed
page 24 processed
page 25 processed
page 26 processed
page 27 processed
page 28 processed
page 29 processed
page 30 processed
page 31 processed
page 32 processed
page 33 processed
page 34 processed
page 35 processed
page 36 processed
page 37 processed
page 38 processed
page 39 processed
page 40 processed
page 41 processed
page 42 processed
page 43 processed
page 44 processed
page 45 processed
page 46 processed
page 47 processed
page 48 processed
page 49 processed
page 50 processed
page 51 processed
page 52 processed
page 53 processed
page 54 processed
page 55 processed
page 56 processed
p

In [3]:
df_info.shape

(294402, 1)

In [4]:
df_info.shape

(294402, 1)

In [5]:
# Сохранение результата
df_info.to_csv('kazakhstan_all_participants_numbers.csv')

### Выгрузка данных о участниках

Ребята решили не запариваться, поэтому что для поставщиков, что для заказчиков сделали одинаковую страницу show_supplier 

In [6]:
df_info = pd.read_csv('kazakhstan_all_participants_numbers.csv', dtype = str).drop(['Unnamed: 0'], axis = 1)

Скачаем только недостающие данные в последней версии файла data_main_info

In [21]:
data_main_info = pd.read_csv('main_Kazakhstan.csv', dtype = str)

In [25]:
ids_add = list(set(df_info['id']) - set(data_main_info['id']))

In [30]:
# Функция для парсинга информации о поставщике по его id на сайте
def parse_supplier_data(participant_id):
    while True:
        try:
            # List of proxy servers from http://spys.one/en/. All ips were checked on fast response prior
            # to main function run. IMPORTANT: TAKE HTTPS SERVERS ONLY, NOT HTTP
            proxy_list = ['157.245.4.19:3128',
             '188.126.45.57:3128', '94.141.60.69:8080', '5.16.13.38:8080',
              '46.73.33.253:8080', '93.175.203.124:56925',
              '35.177.127.142:3128', '167.172.167.174:8080', '212.92.204.54:80',
              '86.110.27.165:3128', '62.173.145.48:3128', '87.103.234.116:3128',
              '80.249.82.44:32004', '86.57.177.8:31022', '181.191.243.110:8080']
            
            url = 'https://goszakup.gov.kz/ru/registry/show_supplier/' + participant_id
            proxy = {'https': random.choice(proxy_list)}
            # If response is not given within 5 seconds, break: timeout=5
            web_page = requests.get(url, proxies=proxy, verify=False, timeout=5)
            # web_page = requests.get(url, verify=False, timeout=5)
            soup = BeautifulSoup(web_page.text, "html.parser")
            # print(participant_id)
            # Вся релевантная информация содержится в таблицах с классом: table table-striped
            all_tables = soup.findAll('table', attrs={'class':'table table-striped'})

            if len(all_tables) == 0:
                continue

            # Здесь распарсим только первые 3 раздела. Последний раздел "контактная информация" сохраним в отдельный 
            # info_contact_info, так как она содержит несколько, заранее не определенное количество строк
            info_main = {}
            for ind_table, table in enumerate(all_tables[0:(len(all_tables)-1)]):
                # В каждой таблице несколько строк: tr
                all_tr = table.findAll('tr')
                # Учтем номер таблицы, чтобы правильно задать названия колонок по разделам
                if ind_table == 0:
                    # В каждой строке два столбца: th и td: th -- название инфы, td -- сама инфа
                    for tr in all_tr:
                        attr = tr.find('th')
                        val = tr.find('td')
                        info_main.update({'Основное.' + attr.text.strip(): val.text.strip()})
                elif ind_table == 1:
                    # В каждой строке два столбца: th и td: th -- название инфы, td -- сама инфа
                    for tr in all_tr:
                        attr = tr.find('th')
                        val = tr.find('td')
                        info_main.update({'Атрибуты.' + attr.text.strip(): val.text.strip()})
                # Бывает, что отсутствует информация о руководителе. В таком случае ind_table < 2. Закептчим этот случай
                elif ind_table == 2:
                    # В каждой строке два столбца: th и td: th -- название инфы, td -- сама инфа
                    for tr in all_tr:
                        attr = tr.find('th')
                        val = tr.find('td')
                        info_main.update({'Руководитель.' + attr.text.strip(): val.text.strip()})
            # Добавим, наконец, информацию о id
            info_main.update({'id': participant_id})
            

#             # Здесь распарсим только раздел "Контактная информация". Вся инфа сидит в tr со второго по последний
#             all_tr = all_tables[(len(all_tables)-1)].findAll('tr')
#             info_contact_info = []
#             # Бывает, что контактная информация отсутствует. Словим эти случаи
#             if len(all_tr) > 1:
#                 for tr in all_tr[1:len(all_tr)]:
#                     # Вся информация содержится в td. Вытащим ее
#                     all_td = tr.findAll('td')
#                     # Сохраним все в df_info_contact_info под соответствующие колонки
#                     info_contact_info.append({'id': participant_id,
#                                             'Страна': all_td[0].text.strip(),
#                                             'КАТО': all_td[1].text.strip(),
#                                             'Полный адрес(рус)': all_td[2].text.strip(),
#                                             'Полный адрес(каз)': all_td[3].text.strip(),
#                                             'Тип адреса': all_td[4].text.strip()})
        except KeyboardInterrupt:
            raise
        except:
#             continue
            print("Sleep")
            time.sleep(20)
        break
    return info_main
#     return [info_main, info_contact_info]

### Unparalleled version

In [33]:
result_add_info = []
for add_info in tqdm.tqdm(enumerate(ids_add), total=len(ids_add)):
    result_add_info.append(parse_supplier_data(add_info[1]))

100%|██████████| 18078/18078 [5:01:59<00:00,  2.28it/s]   


In [41]:
data_main_info = data_main_info.append(result_add_info)

In [42]:
data_main_info.to_csv('main_Kazakhstan_new.csv', index = False)

Paralleled version

In [4]:
# Get the list of suppliers ids
participant_ids = df_info['id'].values

In [6]:
# Initialize 30 processes
p = Pool(30)

In [8]:
# Use multiprocessing to parse data
result_main_info = []
# result_contact_info = []
# for info_main, info_contact_info in tqdm.tqdm(p.imap(parse_supplier_data, participant_ids),
#                                               total=len(participant_ids)):
#     result_main_info.append(info_main)
#     result_contact_info.append(info_contact_info)

for info_main in tqdm.tqdm(p.imap(parse_supplier_data, participant_ids),
                                              total=len(participant_ids)):
    result_main_info.append(info_main)

100%|█████████▉| 74433/74434 [2:41:31<00:00,  7.68it/s]   


In [19]:
p.close()
time.sleep(15)
p.join()

In [9]:
# Turn data_main_info in DataFrame object
data_main_info = pd.DataFrame([x for x in result_main_info])
# Flatten result_contact_info
# flattened_list = [y for info in result_contact_info for y in info]
# data_contact_info = pd.DataFrame(flattened_list)

In [17]:
data_main_info.drop_duplicates(inplace = True)

(276342, 26)

In [18]:
data_main_info.to_csv('main_Kazakhstan.csv', index = False)

In [84]:
# data_contact_info.to_csv('contact_Kazakhstan.csv')